# Example 5: Stopping optimization based on relative improvements

In this advanced example we show how to work with the convergence rate to increae the speed of reaching a solution  (fewer steps). During normal application, the `.auto` method will take a pre-specified number of iterations before stopping. Here, by monitoring the relative improvement in best guess for the maximum response, the algorithm is stopped when the relative improvement falls below a limit `rel_tol`. 


## Considerations for estimating convergence based on relative improvements

Briefly, the Bayesian optimization approach used by this library works by iteratively making a model-driven guess of which data point to sample (determined by the *acquisition function*) and subsequently adjusting its understanding of the hidden function producing the response (the *surrogate model*, which is input to the acquisition function). At each iteration the framework stores the best response found across all sampling points (in the `best_response_value` attribute), but there is no guarantee that the response is better than at previous steps; in fact, it is not uncommon to find that the best response value plateaus for a few consecutive iterations before a sampling point is reached which improves it again.

Using the relative improvement in best response to stop the iterations of the optimization, one needs to be mindful of this plateauing. This example illustrates how to use the framework to address this issue.


## Problem solved in this notebook

In this notebook we will illustrate how to use relative improvement by optimizing this known function 

$$
f(x) = - (6x - 2)^2 \sin{(12 x - 4 )}, \quad x \in [0; 1].
$$

It is known that the function above has its maximum at $x^* = 0.75725$ with a corresponding response of $f(x^*) = 6.02073$; the only covariate is $x$.

We will solve it using the `.auto`-method in and apply conditions on the relative improvement and illustrate their impact on covergence for two different cases

1. set the `rel_tol` argument to stop as soon as the relative improvement between two consecutive steps falls below the user-defined limit
2. use `rel_tol_steps` argument together with `rel_tol` to require that the relative improvement must be below `rel_tol` for `rel_tol_steps` consecutive steps.


## Technical note

Installation of `torch` and `torchvision` (required dependencies) cannot be bundled as part of the `creative_project` installable. This is unfortunate, but a known issue it seems. Therefore these must be installed first, before installing `creative_project`.

In [ ]:
# Preamble

# Install torch and torchvision. Use this link to identify the right versions to install on your system 
# depending on configuration: https://pytorch.org/get-started/locally/
#
#pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
#
# Install creative_project from github (will require authentication with password)
#pip install --user https://github.com/svedel/kre8_core/

In [ ]:
# import
import numpy as np
import pandas as pd
import torch
from creative_project import CreativeProject
import matplotlib.pyplot as plt
%matplotlib inline

Define the function to optimize

In [ ]:
def f(x):
    covar0 = x["covar0"].values
    return -(6*covar0 - 2)**2*np.sin(12*covar0-4)

x = pd.DataFrame({"covar0": np.linspace(0,1)})
plt.figure(figsize=(8, 4))
plt.plot(x["covar0"], f(x))
plt.show()

Set up the problem

In [ ]:
# range of the covariate 'x'
x_exp = 0.5  # start point for x (a priori expectation)
x_min = 0  # minimum allowed value of x
x_max = 1  # maximum allowed value of x
x_input=[(x_exp, x_min, x_max)]

# relative importance limits for convergence
rel_tol = 1e-10
rel_tol_steps = 5

# maximum number of iterations
max_iter = 100

## Solve using just `rel_tol`

First solve the problem using only `rel_tol` limit, requiring only that the relative improvement limit `rel_tol` need to be satisfied for a single step

In [ ]:
# initialize class instance
cc = CreativeProject(covars=x_input, model="SingleTaskGP")

In [ ]:
# solve the problem
cc.auto(response_samp_func=f, max_iter=max_iter, rel_tol=rel_tol)

## Solve using `rel_tol` and `rel_tol_steps`

Solve the same problem using both `rel_tol` and `rel_tol_steps` arguments to require that the relative importance limit is satisfied for `rel_tol_steps` before considering the solution as converged

In [ ]:
# initialize class instance
cc2 = CreativeProject(covars=x_input, model="SingleTaskGP")

In [ ]:
# solve the problem
cc2.auto(response_samp_func=f, max_iter=max_iter, rel_tol=rel_tol, rel_tol_steps=rel_tol_steps)

## Plot and compare solutions

In [ ]:
# define helper function to calculate relative improvement at each solution step
def relative_improvement(creative_class_instance):
    
    # compute the relative differences in response between consecutive iterations
    tmp_array = torch.cat(
            (creative_class_instance.best_response_value[0:-1], creative_class_instance.best_response_value[1:]),
            dim=1).numpy()
    tmp_rel_diff = np.diff(tmp_array, axis=1) / creative_class_instance.best_response_value[1:].numpy()
    
    return tmp_rel_diff

In [ ]:
# calculate the relative differences
rel_diff_cc = relative_improvement(cc)
rel_diff_cc2 = relative_improvement(cc2)

In [ ]:
# plot relative improvements
fx, ax = plt.subplots(1, 2, figsize=(16, 4))
ax1 = ax[0]
ax2 = ax[1]

# first panel: relative improvement
ax1.plot(torch.arange(start=1, step=1, end=cc.best_response_value.shape[0]).numpy(), rel_diff_cc, "*-b", label="one-step relative tolerance limit")
ax1.plot(torch.arange(start=1, step=1, end=cc2.best_response_value.shape[0]).numpy(), rel_diff_cc2, "g", label= str(rel_tol_steps) + "-step relative tolerance limit")
ax1.plot([1, cc2.best_response_value.shape[0]], [rel_tol, rel_tol], "--k", label="limit for convergence")

ax1.set_yscale('log')
ax1.set_title("Maximum number of iterations: " + str(max_iter))
ax1.set_xlabel("Iteration $n$")
ax1.set_ylabel("Relative improvement between iterations $(y_n - y_{n-1})/y_n$")
ax1.legend()

# second panel: best response
ax2.plot(torch.arange(start=0, step=1, end=cc.best_response_value.shape[0]).numpy(), cc.best_response_value.numpy(), "*-b", label="one-step relative tolerance limit")
ax2.plot(torch.arange(start=0, step=1, end=cc2.best_response_value.shape[0]).numpy(), cc2.best_response_value.numpy(), "g", label= str(rel_tol_steps) + "-step relative tolerance limit")

ax2.set_title("Maximum number of iterations: " + str(max_iter))
ax2.set_xlabel("Iteration $n$")
ax2.set_ylabel("Estimate of maximum value $y_n$ at iteration $n$")
ax2.legend()

Notice that few number of iterations is required before reaching the convergence limit in the case where the limit only needs to be reached for at least one iteration (solution with one-step relative improvement tolerance limit, `cc` solution illustrated by blue above). Contrarily, the solution when requiring that the relative improvement limit is maintained for `rel_tol_steps` = 5 consecutive steps leads to better results (green line).

In general, it is advised to require the relative improvement to be below the threshold for at least 3 consecutive steps for the solution to be trustworthy, i.e. set `rel_tol_steps` $\geq 3$.

### Built-in plots of convergence

A method to produce a convergence plot is also included with the framework. This illustrates the convergence for a single optimization task, as can be seen here

In [ ]:
cc2.plot_convergence()

And a method to illustrate the best objective values are also included

In [ ]:
cc2.plot_best_objective()